### Installations and imports

In [ ]:
#!pip install sacremoses
#!pip install transformers
#!pip install datasets
#!pip install evaluate
#!pip install accelerate -U
#!pip install transformers[torch]
#!pip install accelerate==0.20.1
#!pip install transformers==4.17

In [ ]:
import numpy as np
import pandas as pd
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import FlaubertTokenizer, AutoTokenizer
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset
import evaluate

### Load data and first manipulations

In [ ]:
# Load Data
df = pd.read_csv('intent-detection-train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    75 non-null     object
 1   label   75 non-null     object
dtypes: object(2)
memory usage: 1.3+ KB


In [ ]:
# Let's match each label with a unique index
label_to_index = {label : id for id, label in enumerate(df['label'].unique().tolist())}
index_to_label = {id : label for id, label in enumerate(df['label'].unique().tolist())}

# Now we Create the X, y vectors
X = df['text'].tolist()
Y = df['label'].apply(lambda x: label_to_index[x])

In [ ]:
# Initialize the model Autotokenizer from a french model
modelname = 'flaubert/flaubert_base_cased'
tokenizer = AutoTokenizer.from_pretrained(modelname, do_lowercase=False)

In [ ]:
# We train/test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

dataset_train = Dataset.from_list([{'label' : y, 'text':x} for x,y in zip(X_train, y_train)])
dataset_test = Dataset.from_list([{'label' : y, 'text':x} for x,y in zip(X_test, y_test)])

In [ ]:
# Define the tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# We tokenize our train and test datasets
tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True)

Map:   0%|          | 0/67 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [ ]:
# Now we create the Dataset
#tokenized_dataset_train = Dataset.\
#from_list([{'label' : y, 'text':tokenizer.encode(x, padding="max_length", truncation=True)} for x, y in zip(X_train,y_train)])
#tokenized_dataset_test = Dataset.\
#from_list([{'label' : y, 'text':tokenizer.encode(x, padding="max_length", truncation=True)} for x, y in zip(X_test,y_test)])

In [ ]:
# Let's define the metrics to evalute the performance of our classifier
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# Let's load the model
model = AutoModelForSequenceClassification.from_pretrained(modelname, num_labels=9)

In [ ]:
# We set the training arguments
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    learning_rate = 5e-5,
    num_train_epochs = 3
    )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# We build the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train.shuffle(seed=6).select(range(67)),
    eval_dataset=tokenized_dataset_test.shuffle(seed=6).select(range(8)),
    compute_metrics=compute_metrics,
)

In [ ]:
# We make the training procedure
trainer.train()

The following columns in the training set  don't have a corresponding argument in `FlaubertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `FlaubertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 67
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 45


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.719067,0.375000
2,No log,1.670526,0.500000
3,No log,1.539869,0.500000
4,No log,1.583666,0.500000
5,No log,1.596811,0.500000


The following columns in the evaluation set  don't have a corresponding argument in `FlaubertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `FlaubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `FlaubertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `FlaubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `FlaubertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `FlaubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8
  Batch size = 

TrainOutput(global_step=45, training_loss=0.7296897888183593, metrics={'train_runtime': 32.7736, 'train_samples_per_second': 10.222, 'train_steps_per_second': 1.373, 'total_flos': 87539953259520.0, 'train_loss': 0.7296897888183593, 'epoch': 5.0})

### We try leave-two-out train/test procedure to really evaluate this technique.

In [ ]:
# The model
model = AutoModelForSequenceClassification.from_pretrained(modelname, num_labels=9)

In [ ]:
#for i in range(len(X)):
for seed in range(30):
    # We train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.025, random_state=seed)
    #X_train, X_test, y_train, y_test = X[:i]+X[i+2:], [X[i], X[-i]], Y[:i]+Y[i+2:], [Y[i], Y[-i]]

    Split train/test data
    dataset_train = Dataset.from_list([{'label' : y, 'text':x} for x,y in zip(X_train, y_train)])
    dataset_test = Dataset.from_list([{'label' : y, 'text':x} for x,y in zip(X_test, y_test)])

    # We tokenize our train and test datasets
    tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True)
    tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True)

    # We rebuild the model to train it from scratch
    model_copy = AutoModelForSequenceClassification.from_pretrained(modelname, num_labels=9)

    # Trainer
    trainer = Trainer(
        model=model_copy,
        args=training_args,
        train_dataset=tokenized_dataset_train.shuffle(seed=6).select(range(73)),
        eval_dataset=tokenized_dataset_test.shuffle(seed=6).select(range(2)),
        compute_metrics=compute_metrics,
    )

    # Train the model
    trainer.train()

    # The results od the training procedure are printed by the train function

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

loading configuration file https://huggingface.co/flaubert/flaubert_base_cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/0b9ef58865bb61b2a44569c51b24b441c7b6b49ba63c659fc4ad5d61ffa011d6.c03a6cc0529664af7ebd7b4b385954d9cd0071c3d965d9377ab407e2eaa06918
Model config FlaubertConfig {
  "_name_or_path": "flaubert/flaubert_base_cased",
  "amp": 1,
  "architectures": [
    "FlaubertWithLMHeadModel"
  ],
  "asm": false,
  "attention_dropout": 0.1,
  "bos_index": 0,
  "bos_token_id": 0,
  "bptt": 512,
  "causal": false,
  "clip_grad_norm": 5,
  "dropout": 0.1,
  "emb_dim": 768,
  "embed_init_std": 0.02209708691207961,
  "encoder_only": true,
  "end_n_top": 5,
  "eos_index": 1,
  "fp16": true,
  "gelu_activation": true,
  "group_by_size": true,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "id2lang": {
    "

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.872818,0.500000
2,No log,1.805939,0.500000
3,No log,1.929739,0.500000


The following columns in the evaluation set  don't have a corresponding argument in `FlaubertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `FlaubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `FlaubertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `FlaubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `FlaubertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `FlaubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2
  Batch size = 

...

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

loading configuration file https://huggingface.co/flaubert/flaubert_base_cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/0b9ef58865bb61b2a44569c51b24b441c7b6b49ba63c659fc4ad5d61ffa011d6.c03a6cc0529664af7ebd7b4b385954d9cd0071c3d965d9377ab407e2eaa06918
Model config FlaubertConfig {
  "_name_or_path": "flaubert/flaubert_base_cased",
  "amp": 1,
  "architectures": [
    "FlaubertWithLMHeadModel"
  ],
  "asm": false,
  "attention_dropout": 0.1,
  "bos_index": 0,
  "bos_token_id": 0,
  "bptt": 512,
  "causal": false,
  "clip_grad_norm": 5,
  "dropout": 0.1,
  "emb_dim": 768,
  "embed_init_std": 0.02209708691207961,
  "encoder_only": true,
  "end_n_top": 5,
  "eos_index": 1,
  "fp16": true,
  "gelu_activation": true,
  "group_by_size": true,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "id2lang": {
    "

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.888528,0.000000
2,No log,2.590580,0.000000
3,No log,2.790291,0.000000


The following columns in the evaluation set  don't have a corresponding argument in `FlaubertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `FlaubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `FlaubertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `FlaubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `FlaubertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `FlaubertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2
  Batch size = 

In [ ]:
# Accuracy found = 15/(30*2) = 25%
# This technique seems really bad with the data we have (only 75 sentences)
# I believe with more data this would have been the best technique from far.